In [25]:
from pylab import *
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display
import scipy.optimize as opt
import statistics
import os


os.chdir("D:\python")
#Reading the input data into a data frame
df = pd.read_csv("SolarPrediction.csv")

#Getting some useful paramters such as hour,month,year length of day etc. from the given input data and deleting unnecessary columns
df['Time_conv'] =  pd.to_datetime(df['Time'], format='%H:%M:%S')
df['hour'] = pd.to_datetime(df['Time_conv'], format='%H:%M:%S').dt.hour
df['month'] = pd.to_datetime(df['UNIXTime'].astype(int), unit='s').dt.month
df['year'] = pd.to_datetime(df['UNIXTime'].astype(int), unit='s').dt.year
df['total_time'] = pd.to_datetime(df['TimeSunSet'], format='%H:%M:%S').dt.hour - pd.to_datetime(df['TimeSunRise'], format='%H:%M:%S').dt.hour

A= df.to_numpy(dtype=None, copy=False)  
X=A[:,[4,5,6,7,8,12,13,14,15]]

#Normalising the input data
for i in range(9):
 X[:,i]-=np.mean(X[:,i])
 X[:,i]/=np.std(X[:,i])
y=A[:,4].flatten()

#Defining the sigmoid and RELU activation functions and their derivatives(RELU was finally preferred due to better results)
def sigmoiddef(x):
  return 1 / (1 + np.exp(-x))
def dersigmoiddef(x):
 return sigmoiddef(x)-sigmoiddef(x)*sigmoiddef(x)

def reludef(x):
  return max(0.0, x)
def reluderdef(x):
  if x>0:
    return(1)
  else:
    return(0)

sigmoid =np.frompyfunc(sigmoiddef,1,1)
sigmoidder=np.frompyfunc(dersigmoiddef,1,1)
relu =np.frompyfunc(reludef,1,1)
reluder =np.frompyfunc(reluderdef,1,1)


costs=[]
#Defining a function which calculates the cost function of our neural network for a given training set,given set of weights(theta) and regularisation parameter-lamda
def compute_cost(theta,X,y,lamda): 
 z2=np.zeros((5,1))
 a1=np.zeros((9,1))
 a2=np.zeros((5,1))
 a3=np.zeros((1,1))
    
 cost=0

 theta2,b2,theta3,b3=np.split(theta,[45,50,55],axis=0)
 theta2=np.resize(theta2,(5,9))
 b2=np.resize(b2,(5,1))
 theta3=np.resize(theta3,(1,5))
 b3=np.resize(b3,(1,1))
    
 for i in range(10000):
  a1=np.resize((X[i,:]).copy(),(9,1))
  z2=(np.dot(theta2,a1))+b2
  a2=relu(z2)
  a3=theta3.dot(a2)+b3
  cost+=(1/20000)*(a3[0]-y[i])*(a3[0]-y[i]) 

 cost=cost+lamda*np.sum(np.square(theta2))+np.sum(np.square(theta3))
 costs.append(cost)
 return cost

#Defining a forward propogation function which finds the predicted output of the neural network for a given a set of (weights)theta
def forwardprop(theta,X):
 theta2,b2,theta3,b3=np.split(theta,[45,50,55],axis=0)
 theta2=np.resize(theta2,(5,9))
 b2=np.resize(b2,(5,1))
 theta3=np.resize(theta3,(1,5))
 b3=np.resize(b3,(1,1))
 a1=np.resize((X),(9,1))
 z2=(np.dot(theta2,a1))+b2
 a2=relu(z2)
 a3=theta3.dot(a2)+b3
 return(a3[0])

#Defining a function which calculates the gradient vector of our cost function for a given training set,given set of weights(theta) and regularisation parameter-lamda
#I used a backpropogation algorithm to implement this.
def compute_grad(theta, X, y,lamda):
 theta2,b2,theta3,b3=np.split(theta,[45,50,55],axis=0)
 theta2=np.resize(theta2,(5,9))
 b2=np.resize(b2,(5,1))
 theta3=np.resize(theta3,(1,5))
 b3=np.resize(b3,(1,1))
 theta2grad= np.zeros((5,9))
 theta3grad = np.zeros((1,5))
 b2grad = np.zeros((5,1))
 b3grad=np.zeros((1,1))
 
 for i in range(10000):
  a1=np.resize((X[i,:]),(9,1))
  z2=(np.dot(theta2,a1))+b2
  a2=relu(z2)
  a3=np.dot(theta3,a2)+b3
  del3=((a3[0]-y[i])/10000)
  del3=del3[0]
  temp=np.multiply(theta3.T,reluder(z2))
  del2=del3*temp
  theta3grad = theta3grad + np.multiply(del3,a2.T) 
  b3grad = b3grad + del3
  theta2grad = theta2grad + np.dot(del2,a1.T)
  b2grad = b2grad + del2
  grad=np.zeros(56)
 theta3grad =theta3grad+2*lamda*theta3
 theta2grad =theta2grad+2*lamda*theta2
 for s in range(45):
            for j in range(5):
                for k in range(9):
                    grad[s]=theta2grad[j][k]
 for s in range(45,50):
            for j in range(5):
                grad[s]=b2grad[j][0]
                
 for s in range(50,55):
        for j in range(5):
             grad[s]=theta3grad[0][j]
 grad[55]=b3grad[0][0]

 return grad

#Creating an initial randomized theta vector 
INIT_EPSILON=2
initialtheta=np.random.rand(56)* (2 * INIT_EPSILON) - INIT_EPSILON;
initialtheta=np.reshape(initialtheta,(56,1))

y=np.reshape(y,(len(y),1))
y=np.squeeze(y)

#Setting the regularisation parameter lamda as 0.001 (Upon doing a parametric sweep, obtained the best results for this value of lamda)
lamda=0.001

#Using a built-in optimise function from the scipy.optimize module and passing to it the cost function and gradient funtion we defined above.
#The function takes in the initial theta, cost function and gradient function and minimises the cost function and returns the final set of weights: theta
theta=initialtheta
for epoch in range(30):
        theta = (opt.fmin_tnc(func=compute_cost, x0=np.squeeze(theta), fprime=compute_grad, args=(X, y,lamda)))[0]
        
        
error=[];res=[]

#Finding the test set errors
for k in range(10000,20000):		
       error.append(np.abs(forwardprop(theta,X[k,:])-y[k]))
       res.append(np.abs(forwardprop(theta,X[k,:])))

#Printing the mean error through the test set
print('mean error over the test set=',lamda,sum(error)/len(error))
print('variance of error over the test set',lamda,np.std(error))

show()
  

mean error over the test set= 0.001 3.573168115704329
variance of error over the test set 0.001 2.650197879688187
